<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/BlockChain/python_blockchain_training_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%%writefile hash_util.py
import hashlib as hl
import json

def hash_string_256(string):
  return hl.sha256(string).hexdigest()

def hash_block(block):
  #print(block)
  hashable_block = block.__dict__.copy()
  hashable_block['transactions'] = [tx.to_ordered_dict() for tx in hashable_block['transactions']]
  return hash_string_256(json.dumps(hashable_block, sort_keys=True).encode())


Overwriting hash_util.py


In [17]:
%%writefile node.py
from uuid import uuid4
from blockchain import Blockchain
from verification import Verification


class Node:
  def __init__(self):
    self.id = 'Sumith'
    #self.id = str(uuid4())
    self.blockchain = Blockchain(self.id)
    

  #using function for reusable code.
  def get_transaction_value(self):
    """ Retruns the input of the user(a new transaction amount) as a float
    """
    tx_recipient = input('Enter the recipent of the transaction: ') 
    tx_amount = float(input('Your trasaction amount please: ')) 
    return (tx_recipient, tx_amount)
  
  def get_user_choice(self):
    user_input = input('Your Choice : ')
    return user_input

  def print_blockchain_elements(self):
    #use for loop to output the blockchain list console
    for block in self.blockchain.chain:
      print('outputting block : ', block)
    else:
      print('-' * 20)

  def listen_for_input(self):
    waiting_for_input = True
    while waiting_for_input:
      print('Please choose')
      print('1: Add a new transaction value')
      print('2: Mine a new block')
      print('3: Output the blockchain blocks')
      print('4: check transaction validity')
      print('q : Quit ')
      user_choice = self.get_user_choice()
      if user_choice == '1':
        tx_data = self.get_transaction_value()
        recipient, amount = tx_data
        if self.blockchain.add_transaction(recipient, self.id, amount=amount):
          print('Added transaction')
        else:
          print('Tranaction failed')
        print(self.blockchain.get_open_transactions())
      elif user_choice == '2':
        self.blockchain.mine_block()
      elif user_choice == '3':
          self.print_blockchain_elements()
      elif user_choice == '4':
        if Verification.verify_transactions(self.blockchain.get_open_transactions(), self.blockchain.get_balance):
          print("All Transactions are valid")
        else:
          print('There are invalid transaction')
      elif user_choice == 'q':
        waiting_for_input = False
      else:
        print('Input was invalid, please pick a value from the list!')    
      if not Verification.verify_chain(self.blockchain.chain):
        self.print_blockchain_elements()
        print('Invalid block chain!')
        break
      print('Balance of {} : {:6.2f}'.format(self.id, self.blockchain.get_balance()))  
      print('Choice registered !')
    else:
      print('User left')
    print('Done!') 

node = Node()
node.listen_for_input() 


Overwriting node.py


In [18]:
%%writefile printable.py
class Printable:
  def __repr__(self):
    return str(self.__dict__)


Overwriting printable.py


In [19]:
%%writefile block.py
from time import time
from printable import Printable

class Block(Printable):
  def __init__(self, index, previous_hash, transactions, proof, time=time()):
    self.index = index
    self.previous_hash = previous_hash
    self.transactions = transactions
    self.proof = proof
    self.time = time


Overwriting block.py


In [20]:
%%writefile transaction.py
from collections import OrderedDict
from printable import Printable

class Transaction(Printable):
  def __init__(self, sender, recipient, amount):
    self.sender = sender
    self.recipient = recipient
    self.amount = amount



  def to_ordered_dict(self):
      return OrderedDict([('sender', self.sender),
                          ('recipient', self.recipient),
                          ('amount', self.amount)])

Overwriting transaction.py


In [21]:
%%writefile verification.py
from hash_util import hash_string_256, hash_block

class Verification:
  
  @staticmethod
  def valid_proof(transactions, last_hash, proof):
    guess = (str([tx.to_ordered_dict() for tx in transactions]) + str(last_hash) + str(proof)).encode()
    guess_hash = hash_string_256(guess)
    #print(guess_hash)
    return guess_hash[0:2] == '00'

  @classmethod
  def verify_chain(cls, blockchain):
    """verify current blockchain"""
    for (index, block) in enumerate(blockchain):
      if index == 0:
        continue
      if block.previous_hash != hash_block(blockchain[index -1]):
        return False
      if not cls.valid_proof(block.transactions[:-1], block.previous_hash, block.proof):
        print('Prood of work invalid')
        return False
    return True

  @staticmethod
  def verify_transaction(transaction, get_balance):
    sender_balance = get_balance()
    return sender_balance >= transaction.amount

  @classmethod
  def verify_transactions(cls, open_transactions, get_balance):
    return all([cls.verify_transaction(tx, get_balance) for tx in open_transactions])


Overwriting verification.py


In [22]:
%%writefile blockchain.py
# try except block : 
#####################################################
import functools
import hashlib as hl
import json
from collections import OrderedDict
import sys

from hash_util import  hash_block
from block import Block
from transaction import Transaction
from time import time
from verification import Verification

#global variable
MINING_REWARD = 10

class Blockchain:
  def __init__(self, hosting_node_id):
    genesis_block = Block(index = 0,
                        previous_hash= '',
                        transactions = [],
                        proof = 100,
                        time = 0)

    self.chain = [genesis_block]
    self._open_transactions = []
    self.hosting_node = hosting_node_id
    owner = 'Sumith'
    participants = {owner}
    self.load_data()

  @property
  def chain(self):
      return self._chain[:]
  
  @chain.setter
  def chain(self, val):
    self._chain = val

  def get_open_transactions(self):
    return self._open_transactions[:]

  def load_data(self):
    try:
      with open('blockchain.txt', mode='r') as f:
        file_content = f.readlines()

        blockchain = json.loads(file_content[0][:-1])
        updated_blockchain=[]
        for block in blockchain:
          converted_tx = [Transaction(tx['sender'],
                                      tx['recipient'],
                                      tx['amount']) for tx in block['transactions']]
          updated_block = Block(block['index'],
                                block['previous_hash'],
                                converted_tx,
                                block['proof'],
                                block['time'])
          updated_blockchain.append(updated_block)
        self.chain = updated_blockchain
      
        open_transactions = json.loads(file_content[1])
        updated_transactions = []
        for tx in open_transactions:
          updated_transaction = Transaction(tx['sender'],
                                             tx['recipient'],
                                             tx['amount'])
          updated_transactions.append(updated_transaction) 
        self._open_transactions = updated_transactions
    except (IOError, IndexError):                    
      pass
    except ValueError:
      print('Value error!')
    except Exception as exc:
      print("Oops!",exc)
      print("Oops!",sys.exc_info()[0],"occured.")
      sys.exit(1)
    finally:
      print('Cleanup!')

  def save_data(self):
    try :
      with open('blockchain.txt', mode='w') as f:
        saveable_chain = [block.__dict__ for block in [Block(
            block_el.index,
            block_el.previous_hash,
            [tx.__dict__ for tx in block_el.transactions],
            block_el.proof,
            block_el.time
            ) for block_el in self._chain]]

        f.write(json.dumps(saveable_chain))
        f.write('\n')
        saveable_tx = [tx.__dict__ for tx in self._open_transactions]
        f.write(json.dumps(saveable_tx))
    except IOError:
      print('Error saving file!')

  def proof_of_work(self):
    last_block = self._chain[-1]
    last_hash = hash_block(last_block)
    proof = 0
    while not Verification.valid_proof(self._open_transactions, last_hash, proof):
      proof += 1
    return proof

  def get_balance(self):
    participant = self.hosting_node
    tx_sender = [[tx.amount for tx in block.transactions if tx.sender == participant] for block in self._chain]
    open_tx_sender = [tx.amount for tx in self._open_transactions if tx.sender == participant]
    tx_sender.append(open_tx_sender)
    amount_sent = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_sender,0)
    tx_recipient = [[tx.amount for tx in block.transactions if tx.recipient == participant] for block in self._chain]
    amount_received = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_recipient,0)
    return  amount_received - amount_sent


  def get_last_blockchain_value(self):
    """ returns the last value of the current bloackchain """
    #function to return the value
    if len(self._chain) < 1 :
      return None
    return self._chain[-1]

  def add_transaction(self, recipient,sender, amount=1.0):
    transaction = Transaction(sender, recipient, amount)
    if Verification.verify_transaction(transaction, self.get_balance):
      self._open_transactions.append(transaction)
      self.save_data()
      return True
    return False

  def mine_block(self):
    last_block = self._chain[-1]
    hashed_block = hash_block(last_block)
    proof = self.proof_of_work() 
    reward_transaction = Transaction(sender = 'MINING', recipient = self.hosting_node, amount = MINING_REWARD)
    copied_transactions = self._open_transactions[:]
    copied_transactions.append(reward_transaction)
    block = Block(index = len(self._chain),
                        previous_hash = hashed_block,
                        transactions = copied_transactions,
                        proof = proof,
                        time = 0)
    self._chain.append(block)
    self._open_transactions = []
    self.save_data()
    return True




Overwriting blockchain.py


In [0]:
!python node.py

Cleanup!
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
q : Quit 
Your Choice : 2
Balance of Sumith :  20.00
Choice registered !
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
q : Quit 
Your Choice : 1
Enter the recipent of the transaction: we
Your trasaction amount please: 21
Tranaction failed
[]
Balance of Sumith :  20.00
Choice registered !
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
q : Quit 
Your Choice : 1
Enter the recipent of the transaction: we
Your trasaction amount please: 1
Added transaction
[{'sender': 'Sumith', 'recipient': 'we', 'amount': 1.0}]
Balance of Sumith :  19.00
Choice registered !
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
q : Quit 
Your Ch

In [0]:
!rm blockchain.txt

In [0]:
!cat blockchain.txt

cat: blockchain.txt: No such file or directory


In [0]:
2